In [ ]:
from datasets import load_dataset

ds = load_dataset('ape-ds')

In [ ]:
from transformers import AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

checkpoint = ""
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained('my-tokenizer')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
import torch

def make_translations(batch):
    # Tokenize batch at once
    inputs = tokenizer(["translate Tibetan to English: " + text for text in batch['bo']], 
                       return_tensors="pt", padding=True, truncation=True).to('cuda:0')

    # Run batch inference
    with torch.no_grad():  # Disables gradients for efficiency
        outputs = model.generate(inputs.input_ids)

    # Decode all translations at once
    batch['for-post-edit'] = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return batch

# Apply batch processing (adjust batch_size based on memory constraints)
ds = ds.map(make_translations, batched=True, batch_size=16)  # Adjust batch size as needed


In [ ]:
ds.save_to_disk('ape-ds')